# Importing the libraries

In [128]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import xarray as xr
import rioxarray as rxr
import geopandas as gpd
from shapely.geometry import mapping
import pyproj
import pandas as pd
import harmonize as hz

# Importing the dataset

In [178]:
# Create a path to the data directory
path_data = "../data/Raw/"

In [184]:
# Open the dataset list_xy and list_xdimydim
list_xy = xr.open_dataset( path_data + "list_xy.nc", decode_coords='all')
list_xdimydim = xr.open_dataset(path_data + "list_xdimydim.nc", decode_coords='all')



In [185]:
# Subset list_xy on 3 days
list_xy = list_xy.sel(time=slice('2019-01-01', '2019-01-03'))
list_xdimydim = list_xdimydim.sel(time=slice('2019-01-01', '2019-01-03'))

# Explaining the issue while merging the two datasets

In [181]:
# Merge the two datasets by coordinates
cube_fail = xr.combine_by_coords([list_xy, list_xdimydim], combine_attrs='drop_conflicts')
cube_fail

<xarray.Dataset>
Dimensions:            (x: 298, y: 253, time: 3, ydim: 253, xdim: 298)
Coordinates:
  * x                  (x) float64 3.415e+05 3.424e+05 ... 6.157e+05 6.167e+05
  * y                  (y) float64 5.018e+06 5.017e+06 ... 4.786e+06 4.785e+06
  * time               (time) object 2019-01-01 00:00:00 ... 2019-01-03 00:00:00
    crs                int32 0
    band               int32 ...
    spatial_ref        int32 ...
  * ydim               (ydim) float64 5.018e+06 5.017e+06 ... 4.785e+06
  * xdim               (xdim) float64 3.415e+05 3.424e+05 ... 6.167e+05
Data variables:
    ET_500m            (time, y, x) float64 ...
    Fpar_500m          (time, y, x) float64 ...
    density            (y, x) float32 ...
    u10                (time, y, x) float64 ...
    v10                (time, y, x) float64 ...
    t2m                (time, y, x) float64 ...
    tp                 (time, y, x) float64 ...
    FireMask           (time, ydim, xdim) float64 ...
    First_Day          (time, ydim, xdim) float64 ...
    Last_Day           (time, ydim, xdim) float64 ...
    _1_km_16_days_EVI  (time, ydim, xdim) float64 ...
Attributes:
    Conventions:  CF-1.6
    title:        MCD64A1.061 for aid0001
    institution:  Land Processes Distributed Active Archive Center (LP DAAC)
    source:       AppEEARS v3.12
    references:   See README.md

As they should be in the coordinates reference system and the the same dimensions (x=xdim, y=ydim), the two datasets should be merged. However, the merge fails in the sense we have now 5 dimensions (x,y,time,xdim,ydim) instead of 3 (x,,y,time). Let's see why.

## Hypothesis
- (H1) The name of the coordinates are the cause of the issue. Let's rename the coordinates of list_xdimydim to x and y.
- (H1bis) The name of the coordinates are the cause of the issue. Let's rename the coordinates of list_xy to xdim and ydim.
- (H2) The values of the coordinates are the cause of the issue.
- (H4) The grid_mapping attribute is the cause of the issue.
- (H4) The values of the coordinates are the cause of the issue. Let's change the values of the coordinates of list_xdimydim to the values of the coordinates of list_xy.
- (H5) The values of the coordinates and the name are the causes.

## Let's try the easiest hypothesis first: the coordinates names have to match.

### (H1) Let's rename the coordinates of list_xdimydim to x and y.

In [187]:
# Rename the coordinates of list_xdimydim
list_xdimydim_h1 = list_xdimydim.rename({'xdim': 'x', 'ydim': 'y'})

In [188]:
# Merge the two datasets by coordinates
cube_try_1 = xr.combine_by_coords([list_xy, list_xdimydim_h1], combine_attrs='drop_conflicts')
cube_try_1

<xarray.Dataset>
Dimensions:            (x: 360, y: 298, time: 3)
Coordinates:
  * x                  (x) float64 3.415e+05 3.424e+05 ... 6.157e+05 6.167e+05
  * y                  (y) float64 4.785e+06 4.786e+06 ... 5.017e+06 5.018e+06
  * time               (time) object 2019-01-01 00:00:00 ... 2019-01-03 00:00:00
    crs                int32 0
    band               int32 ...
    spatial_ref        int32 ...
Data variables:
    ET_500m            (time, y, x) float64 nan nan nan nan ... nan nan nan nan
    Fpar_500m          (time, y, x) float64 nan nan nan nan ... nan nan nan nan
    density            (y, x) float32 nan nan nan nan nan ... nan nan nan nan
    u10                (time, y, x) float64 nan nan nan nan ... nan nan nan nan
    v10                (time, y, x) float64 nan nan nan nan ... nan nan nan nan
    t2m                (time, y, x) float64 nan nan nan nan ... nan nan nan nan
    tp                 (time, y, x) float64 nan nan nan nan ... nan nan nan nan
    FireMask           (time, y, x) float64 nan nan nan nan ... nan nan nan nan
    First_Day          (time, y, x) float64 nan nan nan nan ... nan nan nan nan
    Last_Day           (time, y, x) float64 nan nan nan nan ... nan nan nan nan
    _1_km_16_days_EVI  (time, y, x) float64 nan nan nan nan ... nan nan nan nan
Attributes:
    Conventions:  CF-1.6
    title:        MCD64A1.061 for aid0001
    institution:  Land Processes Distributed Active Archive Center (LP DAAC)
    source:       AppEEARS v3.12
    references:   See README.md

### (H1bis) Let's rename the coordinates of list_xy to xdim and ydim.

In [189]:
# Rename the coordinates of list_xy
list_xy_h1bis = list_xy.rename({'x': 'xdim', 'y': 'ydim'})

In [191]:
# Merge the two datasets by coordinates
cube_try_1bis = xr.combine_by_coords([list_xy_h1bis, list_xdimydim], combine_attrs='drop_conflicts')
cube_try_1bis

<xarray.Dataset>
Dimensions:            (xdim: 360, ydim: 298, time: 3)
Coordinates:
  * xdim               (xdim) float64 3.415e+05 3.424e+05 ... 6.167e+05
  * ydim               (ydim) float64 4.785e+06 4.786e+06 ... 5.018e+06
  * time               (time) object 2019-01-01 00:00:00 ... 2019-01-03 00:00:00
    crs                int32 0
    band               int32 ...
    spatial_ref        int32 ...
Data variables:
    ET_500m            (time, ydim, xdim) float64 nan nan nan ... nan nan nan
    Fpar_500m          (time, ydim, xdim) float64 nan nan nan ... nan nan nan
    density            (ydim, xdim) float32 nan nan nan nan ... nan nan nan nan
    u10                (time, ydim, xdim) float64 nan nan nan ... nan nan nan
    v10                (time, ydim, xdim) float64 nan nan nan ... nan nan nan
    t2m                (time, ydim, xdim) float64 nan nan nan ... nan nan nan
    tp                 (time, ydim, xdim) float64 nan nan nan ... nan nan nan
    FireMask           (time, ydim, xdim) float64 nan nan nan ... nan nan nan
    First_Day          (time, ydim, xdim) float64 nan nan nan ... nan nan nan
    Last_Day           (time, ydim, xdim) float64 nan nan nan ... nan nan nan
    _1_km_16_days_EVI  (time, ydim, xdim) float64 nan nan nan ... nan nan nan
Attributes:
    Conventions:  CF-1.6
    title:        MCD64A1.061 for aid0001
    institution:  Land Processes Distributed Active Archive Center (LP DAAC)
    source:       AppEEARS v3.12
    references:   See README.md

### Conclusion:
Ok now, we do have only three dimensions, but we can see the size of the dimensions are different, x changed from 298 to 360 and y changed from 253 to 296. It indicates that there should be  something wrong with the coordinates values. Let's check the coordinates values with another analysis.
 (H1) and (H1bis) do not work. the size of the dimension are greater than before. Let's try the second hypothesis.

## (H2) The values of the coordinates are the cause of the issue.

### First let's look at the coordinates values

In [192]:
# Compute the difference of the sum of the coordinates values between the two datasets
diff_x = np.sum(list_xy.coords['x'].values) - np.sum(list_xdimydim.coords['xdim'].values)
diff_y = np.sum(list_xy.coords['y'].values) - np.sum(list_xdimydim.coords['ydim'].values)

In [193]:
# Is diff_x and diff_y equal to 0?
print(diff_x == 0, diff_y == 0)

True True


### Conclusion: We can see the coordinates seems to have the same values. (H2) rejected.

## (H3) The grid_mapping attribute is the cause of the issue.

### Second let's look at the coordinates reference system

In [194]:
# Get the coordinates reference system of the two datasets
crs_xy = list_xy.rio.crs
crs_xdimydim = list_xdimydim.rio.crs

RioXarrayError: Multiple grid mappings exist.

Why do we have the error "Multiple grid mapping exist"? Because the two datasets have two different grid mapping. Let's see what are the grid mapping of the two datasets.

In [195]:
# Get the grid mapping of the two datasets
grid_mapping_xy = list_xy.rio.grid_mapping
grid_mapping_xdimydim = list_xdimydim.rio.grid_mapping

RioXarrayError: Multiple grid mappings exist.

In [196]:
# Check the grid mapping of each variable of the two datasets
list_xy['x'].rio.grid_mapping, list_xy['y'].rio.grid_mapping, list_xy['Fpar_500m'].rio.grid_mapping, list_xy['ET_500m'].rio.grid_mapping, list_xy['density'].rio.grid_mapping


('spatial_ref', 'spatial_ref', 'crs', 'crs', 'spatial_ref')

In [197]:
list_xdimydim['xdim'].rio.grid_mapping, list_xdimydim['ydim'].rio.grid_mapping, list_xdimydim['FireMask'].rio.grid_mapping, list_xdimydim['_1_km_16_days_EVI'].rio.grid_mapping

('spatial_ref', 'spatial_ref', 'crs', 'crs')

We can see the grid mapping of the variables are different. Let's try to write it. We know we have a sinuoidal projection, so we can write it.

In [198]:
# Create the CRS
crs_sinu = rasterio.crs.CRS.from_string(
"+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs")

In [199]:
# Write the grid mapping
list_xy_map = list_xy.rio.write_grid_mapping(crs_sinu, inplace=True)
list_xdimydim_map = list_xdimydim.rio.write_grid_mapping(crs_sinu, inplace=True)

In [200]:
# Check the grid mapping of the two datasets
grid_mapping_xy = list_xy_map.rio.grid_mapping
grid_mapping_xdimydim = list_xdimydim_map.rio.grid_mapping
# Print the grid mapping of the two datasets
print(grid_mapping_xy, grid_mapping_xdimydim)

PROJCS["unknown",GEOGCS["unknown",DATUM["unknown",SPHEROID["unknown",6371007.181,0]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Sinusoidal"],PARAMETER["longitude_of_center",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]] PROJCS["unknown",GEOGCS["unknown",DATUM["unknown",SPHEROID["unknown",6371007.181,0]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Sinusoidal"],PARAMETER["longitude_of_center",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


It worked properly. Let's try to merge the two datasets again.

In [201]:
# Merge the two datasets by coordinates
cube_try_h3 = xr.combine_by_coords([list_xy_map, list_xdimydim_map], combine_attrs='drop_conflicts')

In [202]:
cube_try_h3

<xarray.Dataset>
Dimensions:            (x: 298, y: 253, time: 3, ydim: 253, xdim: 298)
Coordinates:
  * x                  (x) float64 3.415e+05 3.424e+05 ... 6.157e+05 6.167e+05
  * y                  (y) float64 5.018e+06 5.017e+06 ... 4.786e+06 4.785e+06
  * time               (time) object 2019-01-01 00:00:00 ... 2019-01-03 00:00:00
    crs                int32 0
    band               int32 ...
    spatial_ref        int32 ...
  * ydim               (ydim) float64 5.018e+06 5.017e+06 ... 4.785e+06
  * xdim               (xdim) float64 3.415e+05 3.424e+05 ... 6.167e+05
Data variables:
    ET_500m            (time, y, x) float64 ...
    Fpar_500m          (time, y, x) float64 ...
    density            (y, x) float32 ...
    u10                (time, y, x) float64 ...
    v10                (time, y, x) float64 ...
    t2m                (time, y, x) float64 ...
    tp                 (time, y, x) float64 ...
    FireMask           (time, ydim, xdim) float64 ...
    First_Day          (time, ydim, xdim) float64 ...
    Last_Day           (time, ydim, xdim) float64 ...
    _1_km_16_days_EVI  (time, ydim, xdim) float64 ...
Attributes:
    Conventions:  CF-1.6
    title:        MCD64A1.061 for aid0001
    institution:  Land Processes Distributed Active Archive Center (LP DAAC)
    source:       AppEEARS v3.12
    references:   See README.md

## Conclusion:
It did not work neither. Let's try the next hypothesis.

## (H4) The values of the coordinates are the cause of the issue.
We may have missed something in H2. Let's try to compute the absolute difference of the coordinates values between the two datasets and look at the maximum value.

In [203]:
# Compute the absolute difference of the coordinates values between the two datasets and look for the maximum difference
np.abs(list_xy.coords['x'].values -list_xdimydim.coords['xdim'].values).max(), np.abs(list_xy.coords['y'].values - list_xdimydim.coords['ydim'].values).max()

(1.1641532182693481e-10, 9.313225746154785e-10)

There we are !!!!!! We have some really smal differences in the coordinates values. It is due to the calculation while interpolate! ![]
('https://github.com/corteva/rioxarray/issues/298') Let's try to round the coordinates values.

In [204]:
# Let's round the coordinates values
list_xy.coords['x'].values = np.round(list_xy.coords['x'].values, 6)
list_xy.coords['y'].values = np.round(list_xy.coords['y'].values, 6)


ValueError: Cannot assign to the .values attribute of dimension coordinate a.k.a IndexVariable 'x'. Please use DataArray.assign_coords, Dataset.assign_coords or Dataset.assign as appropriate.

In [205]:
# Use assign_coords to assign the coordinates values from the list_xy to the list_xdimydim
list_xdimydim = list_xdimydim.assign_coords(xdim=list_xy.coords['x'].values, ydim=list_xy.coords['y'].values)

In [206]:
# Compute the absolute difference of the coordinates values between the two datasets and look for the maximum difference
np.abs(list_xy.coords['x'].values -list_xdimydim.coords['xdim'].values).max(), np.abs(list_xy.coords['y'].values - list_xdimydim.coords['ydim'].values).max()

(0.0, 0.0)

Now, the coordinates values are the same. Let's try to merge the two datasets again.


In [207]:
# Merge the two datasets by coordinates
cube_try_H4 = xr.combine_by_coords([list_xy, list_xdimydim], combine_attrs='drop_conflicts')
cube_try_H4

<xarray.Dataset>
Dimensions:            (x: 298, y: 253, time: 3, ydim: 253, xdim: 298)
Coordinates:
  * x                  (x) float64 3.415e+05 3.424e+05 ... 6.157e+05 6.167e+05
  * y                  (y) float64 5.018e+06 5.017e+06 ... 4.786e+06 4.785e+06
  * time               (time) object 2019-01-01 00:00:00 ... 2019-01-03 00:00:00
    crs                int32 0
    band               int32 ...
    spatial_ref        int32 ...
  * ydim               (ydim) float64 5.018e+06 5.017e+06 ... 4.785e+06
  * xdim               (xdim) float64 3.415e+05 3.424e+05 ... 6.167e+05
Data variables:
    ET_500m            (time, y, x) float64 ...
    Fpar_500m          (time, y, x) float64 ...
    density            (y, x) float32 ...
    u10                (time, y, x) float64 ...
    v10                (time, y, x) float64 ...
    t2m                (time, y, x) float64 ...
    tp                 (time, y, x) float64 ...
    FireMask           (time, ydim, xdim) float64 ...
    First_Day          (time, ydim, xdim) float64 ...
    Last_Day           (time, ydim, xdim) float64 ...
    _1_km_16_days_EVI  (time, ydim, xdim) float64 ...
Attributes:
    Conventions:  CF-1.6
    title:        MCD64A1.061 for aid0001
    institution:  Land Processes Distributed Active Archive Center (LP DAAC)
    source:       AppEEARS v3.12
    references:   See README.md

### Conclusion:
It dit not work properly. We have the same error. Let's try to use H1 above this: we will also renamed the coordinates so that they have the same name.

## (H5) The coordinates names and the coordinates values are the cause of the issue.

In [209]:
# Let's try again with just a renamming above the matching of the coordinates values previously done
list_xy_renamed = list_xy.rename({'x': 'xdim', 'y': 'ydim'})

In [210]:
# Merge the two datasets by coordinates
cube_try_H5 = xr.combine_by_coords([list_xy_renamed, list_xdimydim], combine_attrs='drop_conflicts')
cube_try_H5

<xarray.Dataset>
Dimensions:            (xdim: 298, ydim: 253, time: 3)
Coordinates:
  * xdim               (xdim) float64 3.415e+05 3.424e+05 ... 6.167e+05
  * ydim               (ydim) float64 5.018e+06 5.017e+06 ... 4.785e+06
  * time               (time) object 2019-01-01 00:00:00 ... 2019-01-03 00:00:00
    crs                int32 0
    band               int32 ...
    spatial_ref        int32 ...
Data variables:
    ET_500m            (time, ydim, xdim) float64 ...
    Fpar_500m          (time, ydim, xdim) float64 ...
    density            (ydim, xdim) float32 ...
    u10                (time, ydim, xdim) float64 ...
    v10                (time, ydim, xdim) float64 ...
    t2m                (time, ydim, xdim) float64 ...
    tp                 (time, ydim, xdim) float64 ...
    FireMask           (time, ydim, xdim) float64 ...
    First_Day          (time, ydim, xdim) float64 ...
    Last_Day           (time, ydim, xdim) float64 ...
    _1_km_16_days_EVI  (time, ydim, xdim) float64 ...
Attributes:
    Conventions:  CF-1.6
    title:        MCD64A1.061 for aid0001
    institution:  Land Processes Distributed Active Archive Center (LP DAAC)
    source:       AppEEARS v3.12
    references:   See README.md

It worked! We have a cube and 3 dimensions and with the good size. Let's try to write it.

## Conclusion
Due to the interpolation the value of the coordinates are not exactly the same. We can use assign_coord() to match perfectly the value of the dimensions of the two datasets. We also need to rename the dimensions so they can merge properly.